In [ ]:
#@title ### Step number 1 - GPU Check.

!nvidia-smi

In [ ]:
#@title ### Step number 2 - Downloading part 1 + Edits
#@markdown If the download speed is too slow try restart

#@markdown Once you're done extracting- check the left side of the web interface until you see a folder 📂 and press it, then go to "content/naifu/" and open requirements.txt and change "pytorch_lightning" to "pytorch_lightning==1.7.7" and just close the note, it should save (alternatively press ctrl+s).

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
   https://bit.ly/3Ujn0JN \
   https://bit.ly/3FJYA8i
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

In [ ]:
#@title ### Step number 3 - Download part 2
#@markdown Wait patiently for the installation to complete

#@markdown Do not restart or panic when it asks you to restart, just continue.

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared

In [ ]:
#@title ### Step number 4 - Running the app
#@markdown On Step 4, it will give you a randomly generated Cloudflare link- this will lead you to the emulated NAI Image Generator that will last until you run out of free google GPU instances or about 50-80 generates. 

#@markdown (Note: Be sure to wait until you see something like https://0.0.0.0:6969/ before clicking Cloudflare link.)

%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969